In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Curtains"""

# regex_pattern = r'(\d+\-?\s?\d+\/\d+\-?\s?Inch)|(\d+ \d+\/\d+\-?\s?Inch\s?x\s?\d+ \d+\/\d+\-?\s?Inch)|(\d+\.?\d*\s?.?Inch)|()'
# regex_pattern = r'(\d+ \d+\/\d+\-?\s?Inch\s?x\s?\d+ \d+\/\d+\-?\s?Inch)|(\d+\.?\d*\s?.?Inch)|()'
# regex_pattern=r'(?i)(\d+ \d+\/\d+\-?\s?Inch\s?x\s?\d+ \d+\/\d+\-?\s?Inch)|(\d+.?Inch x \d+\-Inch)|(\d+\.?\d*\s?.?Inch)|(\d+.?\d+\/\d+\-Inch)|()'
# regex_pattern=r'(?i)(\d+ \d+\/\d+\-?\s?Inch\s?x\s?\d+ \d+\/\d+\-?\s?Inch)|(\d+.?Inch\s?\w? x \d+\-Inch)|(\d+\.?\d*\s?.?Inch)|(\d+.?\d+\/\d+\-Inch)|()'
regex_pattern=r'(?i)(\d+.\d*.Inch.+x.?\d+.?Inch)|(\d+.?\d+\/\d+.?Inch.?x.?\d+.?\d+\/\d+.?Inch)|(\d+.?\d+\/\d+\-Inch)|(\d+\.?\d*\s?.?Inch)|()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

attribute      value    customers_name external_id  \
0        length    0.01 in  bedbathandbeyond    66272168   
1        length    0.01 in  bedbathandbeyond    67614349   
2        length    0.02 in  bedbathandbeyond    69210471   
3        length     0.1 in  bedbathandbeyond    69481838   
4        length     0.1 in  bedbathandbeyond    69481839   
...         ...        ...               ...         ...   
91643    length        n/a  bedbathandbeyond    77495078   
91644    length        n/a  bedbathandbeyond    77495082   
91645    length  Under 24"  bedbathandbeyond      206570   
91646    length  Under 24"  bedbathandbeyond     3284235   
91647    length  Under 24"  bedbathandbeyond     3288472   

                                                                                long_desc  \
0                                   SlipX Solutions® Floor to Ceiling PEVA Liner in Clear   
1                                          Lush Decor Animal Print Blanket Sets in Purple   
2                                                         Therapedic® Waterproof Underpad   
3                                                       Lush Decor Shower Curtain in Gray   
4                                                       Lush Decor Shower Curtain in Navy   
...                                                                                   ...   
91643                          BergHOFF® Leo 64 oz. Covered Glass Pasta Container in Pink   
91644  BergHOFF® Leo Tall Smart Seal Extra Large Food Storage Container with Lid in Green   
91645                                  Otello Honeycomb Pinch Pleat Window Curtain Panels   
91646                                                                Verso Window Valance   
91647                                              Waverly® Lovely Lattice Window Valance   

      resolution  
0       standard  
1       standard  
2       standard  
3       standard  
4       standard  
...          ...  
91643   standard  
91644   standard  
91645   standard  
91646   standard  
91647   standard  

[91648 rows x 6 columns]

In [5]:
# metric='in'
# range_string= """
# """

# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# list_values

# kwp=KeywordProcessor()
# print(kwp.add_keywords_from_list(list_values))
# # Extract Key Words
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts()

In [6]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [7]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute      value    customers_name external_id  \
0        length    0.01 in  bedbathandbeyond    66272168   
1        length    0.01 in  bedbathandbeyond    67614349   
2        length    0.02 in  bedbathandbeyond    69210471   
3        length     0.1 in  bedbathandbeyond    69481838   
4        length     0.1 in  bedbathandbeyond    69481839   
...         ...        ...               ...         ...   
91643    length        n/a  bedbathandbeyond    77495078   
91644    length        n/a  bedbathandbeyond    77495082   
91645    length  Under 24"  bedbathandbeyond      206570   
91646    length  Under 24"  bedbathandbeyond     3284235   
91647    length  Under 24"  bedbathandbeyond     3288472   

                                                                                long_desc  \
0                                   SlipX Solutions® Floor to Ceiling PEVA Liner in Clear   
1                                          Lush Decor Animal Print Blanket Sets in Purple   
2                                                         Therapedic® Waterproof Underpad   
3                                                       Lush Decor Shower Curtain in Gray   
4                                                       Lush Decor Shower Curtain in Navy   
...                                                                                   ...   
91643                          BergHOFF® Leo 64 oz. Covered Glass Pasta Container in Pink   
91644  BergHOFF® Leo Tall Smart Seal Extra Large Food Storage Container with Lid in Green   
91645                                  Otello Honeycomb Pinch Pleat Window Curtain Panels   
91646                                                                Verso Window Valance   
91647                                              Waverly® Lovely Lattice Window Valance   

      resolution matches  
0       standard      []  
1       standard      []  
2       standard      []  
3       standard      []  
4       standard      []  
...          ...     ...  
91643   standard      []  
91644   standard      []  
91645   standard      []  
91646   standard      []  
91647   standard      []  

[91648 rows x 7 columns]

In [8]:
df=df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]

In [9]:
df['matches'].explode().value_counts()

24-Inch                433
18-Inch                338
30-Inch                334
36-Inch                304
16-Inch                 91
                      ... 
50-Inch x 48-Inch        1
35.5-Inch x 48-Inch      1
71-Inch W x 74-Inch      1
37-Inch x 64-Inch        1
30-Inch x 12-Inch        1
Name: matches, Length: 401, dtype: int64

In [10]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
    data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [11]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [12]:
def replace_none(data):
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)
    data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == 'None' else x)

In [13]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")

In [14]:
def min_max(data):
    data['length']=data['length'].astype(float)
    data['width']=data['width'].astype(float)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [15]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split(' ')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [16]:
# error

In [17]:
# x=df['matches']
# x

In [18]:
# x[0:50]

In [19]:
# x.str.split(",",expand=True,)

In [20]:
# x=df
# x['matches']=x['matches'].str.get(0).apply(lambda x:x.lower())
# x

In [21]:
# x[['length', 'width']]=x['matches'].str.split(",",expand=True,)

In [22]:
# error

In [23]:
split_col(df) #might throw an error and have to drop nane

In [24]:
df['width']=df['width'].astype(str).apply(lambda x: 0 if x == 'nan' else x)
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                    matches       length        width  
80147   standard                    40-inch      40-inch            0  
80169   standard                    24-inch      24-inch            0  
80174   standard          36-inch x 72-inch      36-inch      72-inch  
80188   standard  12 1/2-inch x 16 1/4-inch  12 1/2-inch  16 1/4-inch  
80189   standard  10 1/2-inch x 12 1/2-inch  10 1/2-inch  12 1/2-inch  
...          ...                        ...          ...          ...  
91620   standard                    18-inch      18-inch            0  
91621   standard                  19.7-inch    19.7-inch            0  
91622   standard                    18-inch      18-inch            0  
91632   standard                    60-inch      60-inch            0  
91642   standard                    11-inch      11-inch            0  

[3227 rows x 9 columns]

In [25]:
df['length']=df['length'].astype(str)
df['width']=df['width'].astype(str)
df['length']=df['length'].apply(lambda quote: quote.replace('-', ' '))
df['width']=df['width'].apply(lambda quote: quote.replace('-', ' '))
df['length']=df['length'].apply(lambda quote: quote.replace('inch', ''))
df['width']=df['width'].apply(lambda quote: quote.replace('inch', ''))
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                    matches   length    width  
80147   standard                    40-inch      40         0  
80169   standard                    24-inch      24         0  
80174   standard          36-inch x 72-inch      36       72   
80188   standard  12 1/2-inch x 16 1/4-inch  12 1/2   16 1/4   
80189   standard  10 1/2-inch x 12 1/2-inch  10 1/2   12 1/2   
...          ...                        ...      ...      ...  
91620   standard                    18-inch      18         0  
91621   standard                  19.7-inch    19.7         0  
91622   standard                    18-inch      18         0  
91632   standard                    60-inch      60         0  
91642   standard                    11-inch      11         0  

[3227 rows x 9 columns]

In [26]:
df['length']=df['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df['width']=df['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                    matches   length    width  
80147   standard                    40-inch      40         0  
80169   standard                    24-inch      24         0  
80174   standard          36-inch x 72-inch      36       72   
80188   standard  12 1/2-inch x 16 1/4-inch  12 1/2   16 1/4   
80189   standard  10 1/2-inch x 12 1/2-inch  10 1/2   12 1/2   
...          ...                        ...      ...      ...  
91620   standard                    18-inch      18         0  
91621   standard                  19.7-inch    19.7         0  
91622   standard                    18-inch      18         0  
91632   standard                    60-inch      60         0  
91642   standard                    11-inch      11         0  

[3227 rows x 9 columns]

In [27]:
a=df['length'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['length']=c
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                    matches  length    width  
80147   standard                    40-inch    40.0        0  
80169   standard                    24-inch    24.0        0  
80174   standard          36-inch x 72-inch    36.0      72   
80188   standard  12 1/2-inch x 16 1/4-inch    12.5  16 1/4   
80189   standard  10 1/2-inch x 12 1/2-inch    10.5  12 1/2   
...          ...                        ...     ...      ...  
91620   standard                    18-inch    18.0        0  
91621   standard                  19.7-inch    19.7        0  
91622   standard                    18-inch    18.0        0  
91632   standard                    60-inch    60.0        0  
91642   standard                    11-inch    11.0        0  

[3227 rows x 9 columns]

In [28]:
a=df['width'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['width']=c
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                    matches  length  width  
80147   standard                    40-inch    40.0   0.00  
80169   standard                    24-inch    24.0   0.00  
80174   standard          36-inch x 72-inch    36.0  72.00  
80188   standard  12 1/2-inch x 16 1/4-inch    12.5  16.25  
80189   standard  10 1/2-inch x 12 1/2-inch    10.5  12.50  
...          ...                        ...     ...    ...  
91620   standard                    18-inch    18.0   0.00  
91621   standard                  19.7-inch    19.7   0.00  
91622   standard                    18-inch    18.0   0.00  
91632   standard                    60-inch    60.0   0.00  
91642   standard                    11-inch    11.0   0.00  

[3227 rows x 9 columns]

In [29]:
# x=df
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# x['new'] = c
# x

In [30]:
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)

In [31]:
# remove_fractions(df['length'])

In [32]:
# x=df
# a=eval(x['length'][3:4].astype(str))
# # numbers = eval(a)

In [33]:
# float(sum(Fraction(s) for s in a))

In [34]:
# from fractions import Fraction
# float(sum(Fraction(s) for s in '  12 1/2'.split()))

In [35]:
# x=df
# x['length'] = x['length'].apply(lambda quote: quote.replace('-', ''))
# x['length'] = x['length'].apply(lambda quote: quote.replace('inch', ''))
# # x['length'] = x['length'].apply(lambda x: x.str)
# x

In [36]:
# x[['first', 'second']]=x['length'].str.split("/",expand=True,)
# x[['third', 'fourth']= x['first'].split()
# result = x['third'] + (float(x['first'])/float(x['second']))
# result

In [37]:
# num,den = s.split( '/' )
# wh, num = num.split()
# result = wh + (float(num)/float(den))

In [38]:
# a=x['length'][3:4]
# a.str.split(' ')
# b= [[" ".join(tag.split(" ")[:2]) , " ".join(tag.split(" ")[2:])] for tag in a][0]
# b

In [39]:
# c=b[1]
# c.split('/')

In [40]:
# b[0]
# b

In [41]:
# a.apply(lambda x: sum(Fraction(x.split('/'))))

In [42]:
# float(sum(Fraction(s) for s in x['length'].split()))
# x['length']=x['length'].apply(lambda a: float(sum(Fraction(a.split()))))

In [43]:
# for i in x['length']:
#     if x['length'][i].find('/') > -1:
#         float(sum(Fraction(s) for s in x['length'][i].split()))
#     else:
#         x['length']

In [44]:
# float(sum(Fraction(s) for s in x['length'][i].split()))
# .apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))

In [45]:
# def fraction_combo(numbers):
#     if numbers.find('/') > -1:
#         numbers.apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))
#         #numbers = literal_eval(numbers)
#         #inte, fract = numbers.split(' ')
#         #nom, den = fract.split('/')
#         #tmp = float(nom)/float(den)
#         #num = int(inte) + tmp
#         return str(numbers)
#     else:
#         return numbers

In [46]:
# a=x['length']

In [47]:
# remove_fractions(df)

In [48]:
# x['length'] = x['length'].apply(lambda x: remove_fractions(x))
# x.head

In [49]:
# x['length'] = x['length'].apply(lambda x: x.split(' '))
# x.head()

In [50]:
min_max(df) #error base 10 when there is only a value for max and not min

In [51]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [52]:
na_to_gsheet(df)

succesfully saved to worksheet:`n/a` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


In [53]:
# # range function builder
# string = range_string.replace(metric, '')
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

In [54]:
df

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                    matches  length  width    Max   Min  
80147   standard                    40-inch    40.0   0.00  40.00   0.0  
80169   standard                    24-inch    24.0   0.00  24.00   0.0  
80174   standard          36-inch x 72-inch    36.0  72.00  72.00  36.0  
80188   standard  12 1/2-inch x 16 1/4-inch    12.5  16.25  16.25  12.5  
80189   standard  10 1/2-inch x 12 1/2-inch    10.5  12.50  12.50  10.5  
...          ...                        ...     ...    ...    ...   ...  
91620   standard                    18-inch    18.0   0.00  18.00   0.0  
91621   standard                  19.7-inch    19.7   0.00  19.70   0.0  
91622   standard                    18-inch    18.0   0.00  18.00   0.0  
91632   standard                    60-inch    60.0   0.00  60.00   0.0  
91642   standard                    11-inch    11.0   0.00  11.00   0.0  

[3227 rows x 11 columns]

In [55]:
matches=df
matches['Max']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches

attribute value    customers_name external_id  \
80147    length   n/a  bedbathandbeyond    10133874   
80169    length   n/a  bedbathandbeyond    10664608   
80174    length   n/a  bedbathandbeyond    10849217   
80188    length   n/a  bedbathandbeyond    11526870   
80189    length   n/a  bedbathandbeyond    11526888   
...         ...   ...               ...         ...   
91620    length   n/a  bedbathandbeyond    69708380   
91621    length   n/a  bedbathandbeyond    69708381   
91622    length   n/a  bedbathandbeyond    69708382   
91632    length   n/a  bedbathandbeyond    77360738   
91642    length   n/a  bedbathandbeyond    77465868   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                    matches  length  width    Max   Min  
80147   standard                    40-inch    40.0   0.00     40   0.0  
80169   standard                    24-inch    24.0   0.00     24   0.0  
80174   standard          36-inch x 72-inch    36.0  72.00     72  36.0  
80188   standard  12 1/2-inch x 16 1/4-inch    12.5  16.25  16.25  12.5  
80189   standard  10 1/2-inch x 12 1/2-inch    10.5  12.50   12.5  10.5  
...          ...                        ...     ...    ...    ...   ...  
91620   standard                    18-inch    18.0   0.00     18   0.0  
91621   standard                  19.7-inch    19.7   0.00   19.7   0.0  
91622   standard                    18-inch    18.0   0.00     18   0.0  
91632   standard                    60-inch    60.0   0.00     60   0.0  
91642   standard                    11-inch    11.0   0.00     11   0.0  

[3227 rows x 11 columns]

In [56]:
matches[curation_col] = matches['Max'].astype(str)+' in'
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

attribute value    customers_name  external_id  \
80147    length   n/a  bedbathandbeyond  ="10133874"   
80169    length   n/a  bedbathandbeyond  ="10664608"   
80174    length   n/a  bedbathandbeyond  ="10849217"   
80188    length   n/a  bedbathandbeyond  ="11526870"   
80189    length   n/a  bedbathandbeyond  ="11526888"   
...         ...   ...               ...          ...   
91620    length   n/a  bedbathandbeyond  ="69708380"   
91621    length   n/a  bedbathandbeyond  ="69708381"   
91622    length   n/a  bedbathandbeyond  ="69708382"   
91632    length   n/a  bedbathandbeyond  ="77360738"   
91642    length   n/a  bedbathandbeyond  ="77465868"   

                                                                                    long_desc  \
80147                   Zenith® TwistTight® 40-Inch Premium Tension Stall Shower Rod in White   
80169           Warp Brothers® Lifeliner® 24-Inch x 10-Foot Non-Adhesive Shelf liner in White   
80174       Redi Shade 36-Inch x 72-Inch Light Filtering Cordless Paper Window Shade in White   
80188                            SALT™ 12 1/2-Inch x 16 1/4-Inch Sink Protector Rack in White   
80189                            SALT™ 10 1/2-Inch x 12 1/2-Inch Sink Protector Rack in White   
...                                                                                       ...   
91620                         La Crosse Clock Company Louisa 18-Inch Wall Clock in Slate Grey   
91621                       La Crosse Clock Company Harper 19.7-Inch Wall Clock in Wood/Cream   
91622  La Crosse Clock Company Light-Up Dial 18-Inch Indoor/Outdoor Wall Clock in Brown/White   
91632                     Fanimation Drone 60-Inch Indoor/Outdoor Ceiling Fan in Glossy White   
91642                                            Ballarini Professionale 11-Inch Aluminum Lid   

      resolution                    matches  length  width    Max   Min  \
80147   standard                    40-inch    40.0   0.00     40   0.0   
80169   standard                    24-inch    24.0   0.00     24   0.0   
80174   standard          36-inch x 72-inch    36.0  72.00     72  36.0   
80188   standard  12 1/2-inch x 16 1/4-inch    12.5  16.25  16.25  12.5   
80189   standard  10 1/2-inch x 12 1/2-inch    10.5  12.50   12.5  10.5   
...          ...                        ...     ...    ...    ...   ...   
91620   standard                    18-inch    18.0   0.00     18   0.0   
91621   standard                  19.7-inch    19.7   0.00   19.7   0.0   
91622   standard                    18-inch    18.0   0.00     18   0.0   
91632   standard                    60-inch    60.0   0.00     60   0.0   
91642   standard                    11-inch    11.0   0.00     11   0.0   

       Q:length  
80147     40 in  
80169     24 in  
80174     72 in  
80188  16.25 in  
80189   12.5 in  
...         ...  
91620     18 in  
91621   19.7 in  
91622     18 in  
91632     60 in  
91642     11 in  

[3227 rows x 12 columns]

In [57]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [58]:
error----Switch Max and Min when doing length and width

SyntaxError: invalid syntax (<ipython-input-58-7509e0f4fe46>, line 1)

In [59]:
gc = gcloud.Gsheets()

In [60]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [61]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [62]:
looks_good('Bed Bath & Beyond', attribute, df, matches)